In [1]:
import numpy as np
import csv as csv
import numpy as np

import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
#from keras.models import load_model
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split
from many_net import  many_net_odcv_hd, many_net_only_diff_cont_varc_2d_4_3d, many_net_only_diff_cont_varc_dim

Using TensorFlow backend.


In [2]:
os.getcwd()

'/Users/stefabo/Documents/GitHub/andi_for_organizers'

Importing the data

In [2]:
import csv
trajs_from_files = csv.reader(open('challenge_for_scoring/task2.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

# 1d case

Loading the models and specifiying the trajectory length on which they have been trained (centers)

In [ ]:
net_centers1d=[25,65,125, 225,425,825]
meta_model1d=[]
for i in net_centers1d: #NB I should go the whole way to 925 and use 25 and 60
    m=load_model('nets/task2/1d/Task2_1D_recdout_'+str(i)+'.h5')
    #m=tf.keras.models.load_model('Model_1D_recdout_'+str(i)+'.h5')


    meta_model1d.append(m)
    

Predictions

In [5]:
a1, predictions1d=many_net_odcv_hd(meta_model1d,validation[0],
                 centers=net_centers1d,
                 dim=1,thr=1e-12,skip=[],min_tr=0,max_tr=1000)

In [9]:
predictions1d

array([8.5265920e-02, 9.0859383e-01, 1.5936449e-06, 6.1372323e-03,
       1.4274365e-06, 1.1394274e-05, 5.5049435e-07, 9.9997699e-01,
       1.5846100e-08, 1.0975315e-05, 1.6210866e-01, 8.2626069e-01,
       8.0220543e-06, 1.1601026e-02, 2.1688342e-05, 9.2897199e-06,
       1.0595046e-04, 9.9986696e-01, 3.5106882e-07, 1.7454886e-05,
       7.2737688e-01, 2.5978664e-04, 3.2875784e-02, 1.0884888e-04,
       2.3937869e-01], dtype=float32)

In [6]:
predictions1d=predictions1d.reshape((len(validation[0]),5))

In [7]:
np.save('predictions/task2/task2_predictions1d_more_nets.npy',predictions1d)

# 2d case

In [3]:
net_centers2d=[25,125, 225,425]
meta_model2d=[]
for i in net_centers2d: #NB I should go the whole way to 925 and use 25 and 60
    m=load_model('nets/task2/2d/Task2_2D_checkpoint_'+str(i)+'.h5')
    #m=tf.keras.models.load_model('Model_1D_recdout_'+str(i)+'.h5')


    meta_model2d.append(m)
    

In [16]:
meta_model1d[3].layers[0].input_shape[-1]

4

Predictions on 2d data using nets trained on 2d data

In [ ]:
pred_2d_u2d=many_net_only_diff_cont_varc_dim(meta_model2d,validation[1],
                                             centers=net_centers2d,dim=2)

In [ ]:
predictions2d_u2d=pred_2d_u2d.reshape((len(validation[1]),5))

In [ ]:
np.save('predictions/task2/task2_predictions2d_using2d_CORRECTED.npy',predictions2d_u2d)

Predictions on 2d data using nets trained on 1d data, it won't work well on LW

In [ ]:
a, predictions2d_u1d=many_net_odcv_hd(meta_model1d,validation[1],
                 centers=net_centers1d,
                 dim=2,thr=1e-12,skip=[],min_tr=0,max_tr=1000)

In [ ]:
predictions2d_u1d=predictions2d_u1d.reshape((len(validation[1]),5))

np.save('predictions/task2/task2_predictions2d_u1d_newest.npy',predictions2d_u1d)

# 3d case

In [ ]:
a3, predictions3d=many_net_odcv_hd(meta_model1d,validation[2],
                 centers=net_centers1d,
                 dim=3,thr=1e-12,skip=[],min_tr=0,max_tr=1000)

In [ ]:
predictions3d=predictions3d.reshape((len(validation[2]),5))

In [ ]:
np.save('predictions/task2/task2_predictions3d_newest.npy',predictions3d)

In [6]:
predictions3d_u1d_newest = np.load('task2_predictions3d_newest.npy')

In [7]:
predictions3d_u1d_int = np.load('task2_predictions3d_u1d_more_nets.npy')

In [8]:
np.count_nonzero(predictions3d_u1d_newest-predictions3d_u1d_int)

0

3d using 1d with centers

In [4]:
net_centers1d

[25, 65, 125, 225, 425, 825]

In [5]:
a3, predictions3d_u1d=many_net_odcv_hd(meta_model1d,validation[2],
                 centers=net_centers1d,
                 dim=3,thr=1e-12,skip=[],min_tr=0,max_tr=1000)

In [6]:
predictions3d_u1d=predictions3d_u1d.reshape((len(validation[2]),5))

In [7]:
np.save('predictions/task2/task2_predictions3d_u1d_more_nets.npy',predictions3d_u1d)

3d using combination of 2d trajectories

In [ ]:
predictions3d_u2d = many_net_only_diff_cont_varc_2d_4_3d(meta_model2d,validation[2],
                 centers=net_centers2d,
                 skip=[],min_tr=0,max_tr=1000)

In [ ]:
predictions3d_u2d=predictions3d_u2d.reshape((len(validation[2]),5))

In [ ]:
np.save('predictions/task2/task2_predictions3d_u2d_CORRECTED.npy',predictions3d_u2d)